In [2]:
# loading libraries
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus# sample text for performing tokenization
from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences
import re


In [3]:
# pre-prepare
# nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hackartist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/hackartist/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def generation(start_date, end_date):
    month = start_date % 100
    end_month = end_date % 100
    year = int(start_date / 100)
    date = start_date
    lines = ""
    while date <= end_date:
        filename = "en/{0}-{1:02d}.txt".format(year,month)
        f = open(filename,"r")
        lines = lines + f.read()
        f.close()

        month = month + 1
        if month == 13:
            month = 1
            year = year + 1
        date = (year*100) + month
    return lines

# text = generation(201604, 201605)
# len(text)

In [6]:
# manipulating data
def manipulation(text):
    lines = [ l for l in text.split('\n') ]
    lines = ' '.join(lines)
    lines = re.sub("(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"," ",lines.lower())
    lines = re.sub("[^a-zA-Z]"," ",lines.lower())
    lines = re.sub("[ ]+"," ",lines.lower())
    lines = re.sub("smart contract","smartcontract",lines.lower())
    lines = re.sub("smart contracts","smartcontract",lines.lower())
    lines = re.findall('\w{2,}', lines)
    lines = ' '.join([x for x in lines])
    return lines

# text = manipulation(text)
# len(text)

In [7]:
from nltk import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer

def tokenization(text):
    # pst = PorterStemmer()
    a = set(stopwords.words('english'))
    token = [ x for x in word_tokenize(text.lower()) if x not in a ]
    # token = [ pst.stem(x) for x in token ]

    return token

# tokens = tokenization(text)
# tokens

In [8]:
# getting frequency by word
from nltk.probability import FreqDist

def frequency(tokens):
    ignore_words = ["dao","http","https", "rt", "via", "amp", "great", "good", "back", "con","get","go","based","become","today","like"]
    tokens = [x for x in tokens if x not in ignore_words]
    fdist = FreqDist(tokens)
    fdist1 = fdist.most_common(60)
    return fdist1
# fdist = frequency(tokens)
# fdist

In [9]:
import csv

def text_mining(start_date, end_date):
    text = generation(start_date, end_date)
    text = manipulation(text)
    tokens = tokenization(text)

    return frequency(tokens)

def write_csv(filename, data):
    f=open(filename, "w")
    writer=csv.writer(f)
    writer.writerow(["keyword","count"])
    [ writer.writerow([x,y]) for (x,y) in data ]
    f.close()

def mining_and_write(filename, start_date, end_date):
    write_csv(filename, text_mining(start_date, end_date))

# gen1 = mining_and_write('gen1.csv',201604,201801)
# gen1
# gen2 = mining_and_write('gen2.csv',201802,202003)
# gen2
# gen3 = mining_and_write('gen3.csv',202004,202212)
# gen3

In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/hackartist/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


NameError: name 'documents' is not defined

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [1]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

NameError: name 'bow_corpus' is not defined

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))